# Tunning Logistic and NN model

In [9]:
# load package
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

In [10]:
# load the data
anomoly  = False
# load data either by anomoly or not
if anomoly:
    X=pd.read_csv("https://raw.githubusercontent.com/KelvinYQC/msia420PA_project/main/Data/with_anomaly.csv")
else:
    X=pd.read_csv("https://raw.githubusercontent.com/KelvinYQC/msia420PA_project/main/Data/without_anomaly.csv")
y = X['booking_status']
X.drop(['booking_status'], axis = 1, inplace = True)


# Baseline model-- Logistic Regression


## Modeling

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.shape


(21239, 22)

In [12]:
LR_model = LogisticRegression(random_state=0).fit(X_train, y_train)


/opt/homebrew/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
# score = correct predictions / total number of data
score = LR_model.score(X_test, y_test)
print(score)


0.7965777650320237


In [14]:
y_pred = LR_model.predict(X_test)
# y_pred_prob = LR_model.predict_proba(X_test)
# y_pred_prob
# print(roc_auc_score(y, LR_model.predict_proba(X_test)[:, 1]))

In [15]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.82      0.89      0.86      7076
           1       0.73      0.59      0.65      3385

    accuracy                           0.80     10461
   macro avg       0.77      0.74      0.75     10461
weighted avg       0.79      0.80      0.79     10461



## Logistic regression with ridge

In [16]:
from sklearn.linear_model import RidgeClassifierCV
LR_ridge = RidgeClassifierCV(alphas=[1e-3, 1e-2, 1e-1, 1]).fit(X_train, y_train)
LR_ridge.score(X_test, y_test)

0.8032692859191282

In [17]:
y_pred_ridge = LR_ridge.predict(X_test)
y_pred_ridge

array([0, 1, 0, ..., 0, 0, 0])

In [18]:
print(classification_report(y_test, y_pred_ridge))


              precision    recall  f1-score   support

           0       0.82      0.90      0.86      7076
           1       0.75      0.59      0.66      3385

    accuracy                           0.80     10461
   macro avg       0.78      0.75      0.76     10461
weighted avg       0.80      0.80      0.80     10461



# Second Model: Neural Network

In [19]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_standardize = scaler.fit_transform(X_train)
X_test_standardize = scaler.fit_transform(X_test)



In [20]:
nn1 = MLPClassifier(solver='lbfgs', 
                    # alpha=1e-5,
                    # hidden_layer_sizes=(6,), 
                    random_state=123)
nn1.fit(X_train_standardize,y_train)
y_predNN = nn1.predict(X_test_standardize)

/opt/homebrew/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [21]:
print(classification_report(y_test,y_predNN))

              precision    recall  f1-score   support

           0       0.89      0.91      0.90      7076
           1       0.80      0.76      0.78      3385

    accuracy                           0.86     10461
   macro avg       0.84      0.83      0.84     10461
weighted avg       0.86      0.86      0.86     10461



In [22]:
params = {'hidden_layer_sizes': [(30,),(50,),(70,),(100,)],
         'learning_rate_init': [0.0001, 0.001,0.01, 0.1,1]}
nn_model = MLPClassifier()


In [ ]:
gs_nn1 = GridSearchCV(nn_model,
                      param_grid=params,
                      scoring='roc_auc',
                      cv=10)
gs_nn1.fit(X_train_standardize,y_train)
print(gs_nn1.best_params_)

/opt/homebrew/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn/neural_network/_multi

In [ ]:
gs_knn_pred = gs_nn1.predict(X_test_standardize)

In [ ]:
print(classification_report(y_test,gs_knn_pred))

In [ ]:
print('The final auc score for NN is: ')
print(round(roc_auc_score(y_test, gs_nn1.predict_proba(X_test_standardize)[:, 1]), 3))